In [1]:
library(readxl)
library(dplyr)
library(tidyr)



Anexando pacote: ‘dplyr’


Os seguintes objetos são mascarados por ‘package:stats’:

    filter, lag


Os seguintes objetos são mascarados por ‘package:base’:

    intersect, setdiff, setequal, union




# N=250

In [2]:
file_names <- c("BCF(no_pi_hat)_N250.xlsx", "BCF_N250.xlsx", "BCF(pi_oracle)_N250.xlsx")
model_names <- c("BCF(no_pi_hat)", "BCF(pi_hat)", "BCF(pi_oracle)")

metrics <- c("CATE_RMSE", "CATE_MAE", "CATE_MAPE", "CATE_cover", "CATE_len",
             "ATE_RMSE", "ATE_MAE", "ATE_MAPE", "ATE_cover", "ATE_len", "pi_RMSE", "pi_MAE")


# Function to read data and calculate summary statistics
read_and_summarize <- function(file_name, model_name) {
  df <- read_excel(file_name)
  means <- df %>% summarise(across(all_of(metrics), mean)) %>%
    pivot_longer(cols = everything(), names_to = "metric", values_to = "mean") %>%
    mutate(model = model_name)
  sds <- df %>% summarise(across(all_of(metrics), sd)) %>%
    pivot_longer(cols = everything(), names_to = "metric", values_to = "sd") %>%
    mutate(model = model_name)
  return(list(means = means, sds = sds))
}

# Apply the function to each file
summary_stats_list <- lapply(1:length(file_names), function(i) read_and_summarize(file_names[i], model_names[i]))

# Combine means into a single data frame
mean_df_long <- bind_rows(lapply(summary_stats_list, function(x) x$means))

# Pivot the means data frame to the desired wide format
mean_table <- mean_df_long %>%
  pivot_wider(names_from = metric, values_from = mean) %>%
  select(Model = model, all_of(metrics))

# Combine standard deviations into a single data frame
sd_df_long <- bind_rows(lapply(summary_stats_list, function(x) x$sds))

# Pivot the standard deviations data frame to the desired wide format
sd_table <- sd_df_long %>%
  pivot_wider(names_from = metric, values_from = sd) %>%
  select(Model = model, all_of(metrics))

# Create a table with mean +/- standard deviation
mean_sd_table <- data.frame(Metric = metrics)
for (i in 1:length(model_names)) {
  mean_values <- mean_table %>% filter(Model == model_names[i]) %>% select(-Model) %>% unlist()
  sd_values <- sd_table %>% filter(Model == model_names[i]) %>% select(-Model) %>% unlist()
  mean_sd_table[[model_names[i]]] <- paste0(sprintf("%.4f", mean_values), " +/- ", sprintf("%.4f", sd_values))
}

# Print the tables
cat("Table with Mean +/- Standard Deviation:\n")
print(mean_sd_table)


# Perform Welch's t-tests between BCF(no_pi_hat) and BCF(pi_hat)
cat("\nWelch's t-test between BCF(no_pi_hat) and BCF(pi_hat):\n")

df_no_pi_hat <- read_excel("BCF(no_pi_hat)_N250.xlsx")
df_pi_hat <- read_excel("BCF_N250.xlsx")

metrics <- c("CATE_RMSE", "CATE_MAE", "CATE_MAPE", "CATE_cover", "CATE_len",
             "ATE_RMSE", "ATE_MAE", "ATE_MAPE", "ATE_len", "pi_RMSE", "pi_MAE")

for (metric in metrics) {
  test_result <- t.test(df_no_pi_hat[[metric]], df_pi_hat[[metric]], var.equal = FALSE)
  cat(sprintf("%s: t = %.4f, df = %.2f, p-value = %.4f\n",
              metric, test_result$statistic, test_result$parameter, test_result$p.value))
  if (test_result$p.value < 0.05) {
    cat(sprintf("The difference in %s between BCF(no_pi_hat) and BCF(pi_hat) is statistically significant.\n", metric))
  } else {
    cat(sprintf("The difference in %s between BCF(no_pi_hat) and BCF(pi_hat) is not statistically significant.\n", metric))
  }
}

Table with Mean +/- Standard Deviation:
       Metric    BCF(no_pi_hat)       BCF(pi_hat)    BCF(pi_oracle)
1   CATE_RMSE 0.2617 +/- 0.0819 0.2820 +/- 0.0876 0.2725 +/- 0.0830
2    CATE_MAE 0.2216 +/- 0.0783 0.2406 +/- 0.0847 0.2302 +/- 0.0796
3   CATE_MAPE 0.6108 +/- 0.2315 0.6114 +/- 0.1852 0.6122 +/- 0.2140
4  CATE_cover 0.9632 +/- 0.0820 0.9508 +/- 0.1002 0.9459 +/- 0.1124
5    CATE_len 1.1507 +/- 0.1465 1.1648 +/- 0.1603 1.1146 +/- 0.1441
6    ATE_RMSE 0.1671 +/- 0.1180 0.1943 +/- 0.1236 0.1758 +/- 0.1181
7     ATE_MAE 0.1671 +/- 0.1180 0.1943 +/- 0.1236 0.1758 +/- 0.1181
8    ATE_MAPE 0.3349 +/- 0.2364 0.3895 +/- 0.2479 0.3526 +/- 0.2372
9   ATE_cover 0.9600 +/- 0.1969 0.9300 +/- 0.2564 0.9500 +/- 0.2190
10    ATE_len 0.7550 +/- 0.0874 0.8004 +/- 0.0975 0.7465 +/- 0.0859
11    pi_RMSE 0.3657 +/- 0.0034 0.0809 +/- 0.0116 0.0000 +/- 0.0000
12     pi_MAE 0.3611 +/- 0.0035 0.0634 +/- 0.0094 0.0000 +/- 0.0000

Welch's t-test between BCF(no_pi_hat) and BCF(pi_hat):
CATE_RMSE: t = -1.69